In [ ]:
import ee
import geemap.core as geemap
ee.Authenticate()
ee.Initialize(project='ee-guchvygucy')

In [ ]:
import geemap
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Function to apply scale factors
def applyScaleFactors(image):
    opticalBands = (image.select('SR_B.')
                    .multiply(0.0000275)
                    .add(-0.2))
    thermalBands = (image.select('ST_B.*')
                    .multiply(0.00341802)
                    .add(149.0))
    return (image.addBands(opticalBands, None, True)
            .addBands(thermalBands, None, True))

# Define the AOI (Area of Interest) using a FeatureCollection
aoi = ee.FeatureCollection("projects/ee-guchvygucy/assets/Nalgonda")

# Load the Landsat 8 image collection
l8 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
      .filterDate('2017-11-01', '2017-11-30')
      .filterBounds(aoi)
      .map(applyScaleFactors)
      .sort('CLOUD_COVER', True)
      .median())

# Initialize the map
m = geemap.Map()

# Visualization parameters for True Color
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}
m.addLayer(l8.clip(aoi), visualization, 'True Color (432)')

# Add legend for True Color
legend_dict = {
    '0.0': '#000000',
    '0.1': '#808080',
    '0.2': '#FFFFFF'
}
m.add_legend(legend_title='True Color (432)', legend_dict=legend_dict)

# NDVI calculation
ndvi = l8.normalizedDifference(['SR_B5', 'SR_B4'])

# Visualize NDVI
ndviPalette = ['blue', 'white', 'green']
m.addLayer(ndvi.clip(aoi), {'min': -1, 'max': 1, 'palette': ndviPalette}, 'NDVI')

# Add legend for NDVI
legend_dict = {
    '-1': 'blue',
    '0': 'white',
    '1': 'green'
}
m.add_legend(legend_title='NDVI', legend_dict=legend_dict)

# EVI calculation
evi = l8.expression(
    '2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))', {
        'NIR': l8.select('SR_B5'),
        'Red': l8.select('SR_B4'),
        'Blue': l8.select('SR_B2')
    }
)

# Visualize EVI
eviPalette = ['blue', 'white', 'green']
m.addLayer(evi.clip(aoi), {'min': -1, 'max': 1, 'palette': eviPalette}, 'EVI')

# Add legend for EVI
m.add_legend(legend_title='EVI', legend_dict=legend_dict)

# SAVI calculation
savi = l8.expression(
    '((NIR - Red) / (NIR + Red + 0.5))', {
        'NIR': l8.select('SR_B5'),  # Near-Infrared band
        'Red': l8.select('SR_B4')   # Red band
    }
)

# Visualize SAVI
saviPalette = ['blue', 'white', 'green']
m.addLayer(savi.clip(aoi), {'min': -1, 'max': 1, 'palette': saviPalette}, 'SAVI')

# Add legend for SAVI
m.add_legend(legend_title='SAVI', legend_dict=legend_dict)

# MSAVI calculation
msavi = l8.expression(
    '((2 * NIR + 1 - sqrt((2 * NIR + 1)**2 - 8 * (NIR - Red))) / 2)', {
        'NIR': l8.select('SR_B5'),  # Near-Infrared band
        'Red': l8.select('SR_B4')   # Red band
    }
)

# Visualize MSAVI
msaviPalette = ['blue', 'white', 'green']
m.addLayer(msavi.clip(aoi), {'min': -1, 'max': 1, 'palette': msaviPalette}, 'MSAVI')

# Add legend for MSAVI
m.add_legend(legend_title='MSAVI', legend_dict=legend_dict)

# Load a digital elevation model (DEM) dataset
dem = ee.Image("USGS/SRTMGL1_003")

# Compute slope (in degrees)
slope = ee.Terrain.slope(dem)

# Compute aspect (in degrees clockwise from north)
aspect = ee.Terrain.aspect(dem)

# Visualize slope (in degrees)
slopePalette = ['blue', 'yellow', 'orange', 'red']
m.addLayer(slope.clip(aoi), {'min': 0, 'max': 45, 'palette': slopePalette}, 'Slope (degrees)')

# Add legend for Slope
legend_dict = {
    '0': 'blue',
    '15': 'yellow',
    '30': 'orange',
    '45': 'red'
}
m.add_legend(legend_title='Slope (degrees)', legend_dict=legend_dict)

# Visualize aspect (in degrees)
aspectPalette = ['blue', 'yellow', 'orange', 'red']
m.addLayer(aspect.clip(aoi), {'min': 0, 'max': 360, 'palette': aspectPalette}, 'Aspect (degrees)')

# Add legend for Aspect
legend_dict = {
    '0': 'blue',
    '90': 'yellow',
    '180': 'orange',
    '360': 'red'
}
m.add_legend(legend_title='Aspect (degrees)', legend_dict=legend_dict)

# Calculate area
area = (slope.divide(57.2958)
        .tan()
        .add(0.01)
        .pow(-1)
        .multiply(0.0001)
        .resample('bilinear'))

# Calculate the Topographic Index (TI)
twi = area.divide(slope.tan()).log()

# Visualize TWI
twiPalette = ['blue', 'white', 'green']
m.addLayer(twi.clip(aoi), {'min': -1, 'max': 1, 'palette': twiPalette}, 'TWI')

# Add legend for TWI
m.add_legend(legend_title='TWI', legend_dict=legend_dict)

# NDMI calculation
ndmi = l8.normalizedDifference(['SR_B5', 'SR_B6'])

# Visualize NDMI
ndmiPalette = ['blue', 'white', 'green']
m.addLayer(ndmi.clip(aoi), {'min': -1, 'max': 1, 'palette': ndmiPalette}, 'NDMI')

# Add legend for NDMI
m.add_legend(legend_title='NDMI', legend_dict=legend_dict)

# MNDWI calculation
mndwi = l8.normalizedDifference(['SR_B3', 'SR_B6'])

# Visualize MNDWI
mndwiPalette = ['blue', 'white', 'green']
m.addLayer(mndwi.clip(aoi), {'min': -1, 'max': 1, 'palette': mndwiPalette}, 'MNDWI')

# Add legend for MNDWI
m.add_legend(legend_title='MNDWI', legend_dict=legend_dict)

# MSI calculation
msi = l8.expression(
    '((SWIR) / (NIR))', {
        'SWIR': l8.select('SR_B6'),
        'NIR': l8.select('SR_B5')
    }
)

# Visualize MSI
msiPalette = ['blue', 'white', 'green']
m.addLayer(msi.clip(aoi), {'min': -1, 'max': 1, 'palette': msiPalette}, 'MSI')

# Add legend for MSI
m.add_legend(legend_title='MSI', legend_dict=legend_dict)

# Selecting bands for visualization
l8_nir = l8.select('SR_B5')  # Near-Infrared band (B5)
l8_swir = l8.select('SR_B6')  # Shortwave Infrared band (B6)
l8_tir = l8.select('ST_B10')  # Thermal Infrared band (B10)

# Adding layers for visualization
nirPalette = ['black', 'white']
swirPalette = ['black', 'white']
tirPalette = ['black', 'white']

m.addLayer(l8_nir.clip(aoi), {'min': 0, 'max': 0.3, 'palette': nirPalette}, 'NIR')
m.addLayer(l8_swir.clip(aoi), {'min': 0, 'max': 0.3, 'palette': swirPalette}, 'SWIR')
m.addLayer(l8_tir.clip(aoi), {'min': 0, 'max': 300, 'palette': tirPalette}, 'TIR')

# Add legends for NIR, SWIR, and TIR
nir_legend_dict = {
    '0.0': 'black',
    '0.15': 'grey',
    '0.3': 'white'
}
m.add_legend(legend_title='NIR', legend_dict=nir_legend_dict)

swir_legend_dict = {
    '0.0': 'black',
    '0.15': 'grey',
    '0.3': 'white'
}
m.add_legend(legend_title='SWIR', legend_dict=swir_legend_dict)

tir_legend_dict = {
    '0': 'black',
    '150': 'grey',
    '300': 'white'
}
m.add_legend(legend_title='TIR', legend_dict=tir_legend_dict)

m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…